Se descarga el dataset de ATIS empleado en el modelo de referencia.

Este notebook está enfocado en ejecutar la implementación en Pytorch de JointBERT, el cual está presente en la publicación "BERT for Joint Intent Classification and Slot Filling". El objetivo de ello es tomar como referencia los resultados obtenidos por otros modelos para emplearlos como punto de comparación cuando se emplea el dataset ATIS

In [ ]:
! wget https://raw.githubusercontent.com/yvchen/JointSLU/master/data/atis-2.train.w-intent.iob

--2021-11-24 21:37:54--  https://raw.githubusercontent.com/yvchen/JointSLU/master/data/atis-2.train.w-intent.iob
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 808016 (789K) [text/plain]
Saving to: ‘atis-2.train.w-intent.iob’

atis-2.train.w-inte 100%[===================>] 789.08K  --.-KB/s    in 0.05s   

2021-11-24 21:37:54 (16.7 MB/s) - ‘atis-2.train.w-intent.iob’ saved [808016/808016]



Se clona el modelo JointBERT desde Gthub. Lo utilizaremos como referencia utilizando el dataset ATIS. Este es un modelo con la arquitectura Transformer.

In [1]:
!git clone https://github.com/monologg/JointBERT.git

Cloning into 'JointBERT'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 332 (delta 0), reused 1 (delta 0), pack-reused 329
Receiving objects: 100% (332/332), 486.33 KiB | 13.51 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [ ]:
Se instalan las dependencias para ejecutar el modelo

In [2]:
!pip3 install transformers==3.0.2 seqeval==0.0.12 pytorch-crf==0.7.2

     |████████████████████████████████| 769 kB 32.8 MB/s 
     |████████████████████████████████| 3.0 MB 36.0 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
     |████████████████████████████████| 1.2 MB 25.6 MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7434 sha256=14e9189c8429cff82810812644988f19aa0d3bbe0cc653acdca032d17e846876
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


Se cambia a la carpeta donde se encuentra el código del modelo, y se crea la carpeta donde se guardará el modelo luego durante el entrenamiento

In [3]:
% cd JointBERT
! mkdir ./new_model

/content/JointBERT


Se entrena el modelo con un batch size de 32,  5 épocas y con una secuencia de entrada con longitud máxima de 50 palabras (la data para entrenamiento es la misma que la utilizada en el trabajo de referencia). Tambien se indica que cada 50 iteraciones se guarde modelo y cada 100 pasos se realice una validacion con los datos de tipo "dev" (data/atis/dev).

In [4]:
 ! python3 main.py --task atis \
                  --model_type bert \
                  --model_dir atis_model \
                  --do_train \
                  --train_batch_size 32 \
                  --num_train_epochs 5 \
                  --model_dir ./new_model/ \
                  --max_seq_len 50 \
                  --logging_steps 100 \
                  --save_steps 50           

11/25/2021 15:27:42 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpeizg027l
Downloading: 100% 232k/232k [00:00<00:00, 17.8MB/s]
11/25/2021 15:27:42 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
11/25/2021 15:27:42 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
11/25/2021 15:27:42 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/be

Se evalúa el modelo (la data para pruebas es la misma que la utilizada en el trabajo de referencia)

In [5]:
 ! python3 main.py --task atis \
                  --model_type bert \
                  --model_dir atis_model \
                  --do_eval \
                  --model_dir ./new_model/

11/25/2021 18:50:44 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
11/25/2021 18:50:44 - INFO - data_loader -   Loading features from cached file ./data/cached_train_atis_bert-base-uncased_50
11/25/2021 18:50:44 - INFO - data_loader -   Loading features from cached file ./data/cached_dev_atis_bert-base-uncased_50
11/25/2021 18:50:44 - INFO - data_loader -   Loading features from cached file ./data/cached_test_atis_bert-base-uncased_50
11/25/2021 18:50:44 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.715